In [1]:
%matplotlib inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [2]:
md = pd.read_csv('input/movies_metadata.csv')

FileNotFoundError: File b'input/movies_metadata.csv' does not exist

In [3]:
md.isnull().sum()

NameError: name 'md' is not defined

In [4]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

NameError: name 'md' is not defined

In [5]:
md['genres']

NameError: name 'md' is not defined

In [6]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C


NameError: name 'md' is not defined

In [7]:
m = vote_counts.quantile(0.95)
m

NameError: name 'vote_counts' is not defined

In [8]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

NameError: name 'md' is not defined

In [9]:
md['year']

NameError: name 'md' is not defined

In [10]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][
    ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

NameError: name 'md' is not defined

In [11]:
qualified = qualified.sort_values(by='year', ascending=False)
qualified

NameError: name 'qualified' is not defined

In [12]:
def weighted_rating(x):
    print(x)
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)


In [13]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

NameError: name 'qualified' is not defined

In [14]:
qualified = qualified.sort_values('wr', ascending=False)

NameError: name 'qualified' is not defined

In [15]:
qualified.head(15)

NameError: name 'qualified' is not defined

In [16]:
md.apply(lambda x: pd.Series(x['genres']), axis=1).stack()

NameError: name 'md' is not defined

In [17]:
s = md.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level = 1 ,drop=True)
s

NameError: name 'md' is not defined

In [18]:
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

NameError: name 's' is not defined

In [19]:

def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [20]:
build_chart('Romance').head(15)

NameError: name 'gen_md' is not defined

In [21]:
links_small = pd.read_csv('input/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

FileNotFoundError: File b'input/links_small.csv' does not exist

In [22]:
md = md.drop([19730, 29503, 35587])

NameError: name 'md' is not defined

In [23]:
md['id'] = md['id'].astype('int')

NameError: name 'md' is not defined

In [24]:
smd = md[md['id'].isin(links_small)]
smd.isnull().sum()

NameError: name 'md' is not defined

In [25]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['tagline'] + smd['overview']
smd['description'] = smd['description'].fillna('')

NameError: name 'smd' is not defined

In [26]:
print(np.nan + 12)

nan


In [27]:
smd['description']

NameError: name 'smd' is not defined

In [28]:
#문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩한 벡터를 만든다. + 가중치 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])
print(tfidf_matrix)

NameError: name 'smd' is not defined

In [29]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

NameError: name 'tfidf_matrix' is not defined

In [30]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

NameError: name 'smd' is not defined

In [31]:
def recommanedSystem(name):
    idx = indices[name]
    sims_scores = list(enumerate(cosine_sim[idx]))
    sims_scores = sorted(sims_scores, key=lambda x:x[1], reverse=True)
    sims_scores = sims_scores[1:31]
    movie_indices = [i[0] for i in sims_scores]
    return titles.iloc[movie_indices]

In [32]:
recommanedSystem('The Dark Knight')

NameError: name 'indices' is not defined

In [33]:
credits = pd.read_csv('input/credits.csv')
keywords = pd.read_csv('input/keywords.csv')

FileNotFoundError: File b'input/credits.csv' does not exist

In [34]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

NameError: name 'keywords' is not defined

In [35]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

NameError: name 'md' is not defined

In [36]:
smd = md[md['id'].isin(links_small)]

NameError: name 'md' is not defined

In [37]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))


NameError: name 'smd' is not defined

In [38]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [39]:
smd['director'] = smd['crew'].apply(get_director)

NameError: name 'smd' is not defined

In [40]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)

NameError: name 'smd' is not defined

In [41]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


NameError: name 'smd' is not defined

In [42]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

NameError: name 'smd' is not defined

In [43]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x, x, x])


NameError: name 'smd' is not defined

In [44]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

NameError: name 'smd' is not defined

In [45]:
s = s.value_counts()
s[:5]

NameError: name 's' is not defined

In [46]:
s = s[s > 1] #언급 정도가 1개 이상만 사용되도록한다. 

NameError: name 's' is not defined

In [47]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [48]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [49]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x] )
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

NameError: name 'smd' is not defined

In [50]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

NameError: name 'smd' is not defined

In [51]:
for x, i in enumerate(smd['soup']):
    if x == 10:
        break
    print(i)

NameError: name 'smd' is not defined

In [52]:
count = CountVectorizer(analyzer='word', stop_words='english', ngram_range=(1, 2), min_df=0)
count_matrix = count.fit_transform(smd['soup'])
count_matrix

NameError: name 'smd' is not defined

In [53]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

NameError: name 'count_matrix' is not defined

In [54]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

NameError: name 'smd' is not defined

In [55]:
recommanedSystem('The Dark Knight').head(10)

NameError: name 'indices' is not defined

In [56]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:15]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified


In [57]:
improved_recommendations('The Dark Knight')

NameError: name 'indices' is not defined

In [58]:
reader = Reader()

In [59]:
ratings = pd.read_csv('input/ratings_small.csv')
ratings.head()

FileNotFoundError: File b'input/ratings_small.csv' does not exist

In [60]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

NameError: name 'ratings' is not defined

In [61]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

NameError: name 'data' is not defined

In [62]:
trainset = data.build_full_trainset()
svd.train(trainset)

NameError: name 'data' is not defined

In [63]:
ratings[ratings['userId'] == 1]

NameError: name 'ratings' is not defined

In [64]:
svd.predict(1, 310, 3)

AttributeError: 'SVD' object has no attribute 'trainset'

In [65]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [66]:
id_map = pd.read_csv('input/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
print(id_map.head(3))
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
id_map.head(3)

FileNotFoundError: File b'input/links_small.csv' does not exist

In [67]:
id_map

NameError: name 'id_map' is not defined

In [68]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [69]:
hybrid(3, 'The Dark Knight')

NameError: name 'indices' is not defined